In [29]:
import json 

def load_json_file(file_name):
    with open(file_name, 'r') as f:
        return json.load(f)
    
def save_json_file(file_name, data):
    with open(file_name, 'w') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

In [30]:
from pathlib import Path

OLD_VERSION =  Path('json0914') / 'permanentes.json'
NEW_VERSION =  Path('json0929') / 'permanentes.json'

In [31]:
old_data = load_json_file(OLD_VERSION)
new_data = load_json_file(NEW_VERSION)

In [32]:
COMPARE_DB = {}

for registry in new_data:
    chave = registry['key']
    new_meta = registry.pop('meta')
    new_val = registry
    old_val = next(filter(lambda x: x['key'] == chave, old_data), None)
    if old_val is None:
        COMPARE_DB[chave] = {'new': new_val, 'old': None}
        continue
    else:
        old_meta = old_val.pop('meta')
        COMPARE_DB[chave] = {'old': old_val, 'new': new_val}


In [33]:
def show_dict_diff(dict1: dict, dict2: dict, prefix: str = '', level: int = 0):
    for key in dict1:
        if key not in dict2:
            print(f'Key {key} not in dict2')
            continue
        if dict1[key] != dict2[key]:
            if type(dict1[key]) == dict:
                show_dict_diff(dict1[key], dict2[key], prefix + key + '.', level + 1)
            else:
                print(f'Key {key} has different values')
                print("  "*(level + 1) + f'dict1[{prefix}{key}]: {dict1[key]}')
                print("  "*(level + 1) + f'dict2[{prefix}{key}]: {dict2[key]}')

def return_dict_diff(dict1: dict, dict2: dict, prefix: str = '', level: int = 0):
    diff = {}
    for key in dict1:
        if key not in dict2:
            diff[key] = {'d1': dict1[key], 'd2': None}
            continue
        if dict1[key] != dict2[key]:
            if type(dict1[key]) == dict:
                diff[key] = return_dict_diff(dict1[key], dict2[key], prefix + key + '.', level + 1)
            else:
                diff[key] = {'d1': dict1[key], 'd2': dict2[key]}
    for key in dict2:
        if key not in dict1:
            diff[key] = {'d1': None, 'd2': dict2[key]}
            continue
        if dict1[key] != dict2[key]:
            if type(dict1[key]) == dict:
                diff[key] = return_dict_diff(dict1[key], dict2[key], prefix + key + '.', level + 1)
            else:
                diff[key] = {'d1': dict1[key], 'd2': dict2[key]}
    return diff


In [34]:
d1 = {"nome": "Luiz", "meta": {"idade": 29, 'trusth': True}, "teste": {"teste2": {"teste3": 1}}}
d2 = {"nome": "Luiz", "meta": {"idade": 30, 'trusth': True}, "teste": {"teste1": {"teste3": 4}}}
return_dict_diff(d1, d2)

{'meta': {'idade': {'d1': 29, 'd2': 30}},
 'teste': {'teste2': {'d1': {'teste3': 1}, 'd2': None},
  'teste1': {'d1': None, 'd2': {'teste3': 4}}}}

In [35]:
DIFF_DICT = {'modified': {}, 'new': {}}
for key, val in COMPARE_DB.items():
    if val['old'] == None:
        DIFF_DICT['new'][key] = val['new']
    elif val['old'] != val['new']: 
        show_dict_diff(val['new'], val['old'])
        DIFF_DICT['modified'][key] = return_dict_diff(val['new'], val['old'])

Key responsavel has different values
  dict1[responsavel]: Tiago Davi Curi Busarello
  dict2[responsavel]: Daniel Alejandro Ponce Saldias
Key ambiente has different values
  dict1[ambiente]: B114
  dict2[ambiente]: C304
Key edificio has different values
  dict1[edificio]: BNU03 - Bloco Acadêmico B
  dict2[edificio]: BNU07 - Bloco Administrativo C
Key transf_local has different values
    dict1[detalhes.transf_local]: 20/09/2023
    dict2[detalhes.transf_local]: 15/08/2022
Key origem has different values
  dict1[origem]: B114
  dict2[origem]: C304
Key responsavel has different values
  dict1[responsavel]: Rafael dos Reis Abreu
  dict2[responsavel]: Carolina Suelen da Silva
Key ambiente has different values
  dict1[ambiente]: A201
  dict2[ambiente]: B026
Key edificio has different values
  dict1[edificio]: BNU02 - Bloco Acadêmico A
  dict2[edificio]: BNU03 - Bloco Acadêmico B
Key transf_local has different values
    dict1[detalhes.transf_local]: 15/09/2023
    dict2[detalhes.transf_loca

In [36]:
DIFF_DICT

{'modified': {364012: {'responsavel': {'d1': 'Tiago Davi Curi Busarello',
    'd2': 'Daniel Alejandro Ponce Saldias'},
   'ambiente': {'d1': 'B114', 'd2': 'C304'},
   'edificio': {'d1': 'BNU03 - Bloco Acadêmico B',
    'd2': 'BNU07 - Bloco Administrativo C'},
   'detalhes': {'transf_local': {'d1': '20/09/2023', 'd2': '15/08/2022'}},
   'origem': {'d1': 'B114', 'd2': 'C304'}},
  364318: {'responsavel': {'d1': 'Rafael dos Reis Abreu',
    'd2': 'Carolina Suelen da Silva'},
   'ambiente': {'d1': 'A201', 'd2': 'B026'},
   'edificio': {'d1': 'BNU02 - Bloco Acadêmico A',
    'd2': 'BNU03 - Bloco Acadêmico B'},
   'detalhes': {'transf_local': {'d1': '15/09/2023', 'd2': None}},
   'origem': {'d1': 'A201', 'd2': 'B026'}},
  310298: {'responsavel': {'d1': 'Francis Felix Cordova Puma',
    'd2': 'André Vanderlinde da Silva'}},
  310336: {'responsavel': {'d1': 'Francis Felix Cordova Puma',
    'd2': 'André Vanderlinde da Silva'}},
  312848: {'responsavel': {'d1': 'Francis Felix Cordova Puma',
    

In [37]:
save_json_file('diff.json', DIFF_DICT)

In [38]:
import pandas as pd

In [41]:
modificados = pd.DataFrame(DIFF_DICT['modified']).transpose()
novos = pd.DataFrame(DIFF_DICT['new']).transpose()

In [45]:
novos.shape, modificados.shape

((41, 8), (257, 5))